In [7]:
!pip install -q html2text

In [12]:
import logging
import traceback

def setup_logging(log_file: str = "app.log"):
    # Create a custom logger
    logger = logging.getLogger("LinkExtractor")
    logger.setLevel(logging.INFO)  # Capture only INFO and above (INFO, WARNING, ERROR, CRITICAL)

    # Prevent logger duplication
    if not logger.hasHandlers():
        # Create file handler to log to a file
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)  # File handler also captures only INFO and above

        # Create log format with function name included
        log_format = logging.Formatter('%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s')
        file_handler.setFormatter(log_format)

        # Add handler to the logger
        logger.addHandler(file_handler)

    return logger

# Step 2: Set up logging to a file only, suppressing output in the notebook
logger = setup_logging(log_file="notebook_logs.log")

# Step 3: Example usage
logger.info("This is an INFO log message.")
logger.debug("This DEBUG message won't appear in INFO mode.")
logger.warning("This is a WARNING log message.")

In [13]:
from typing import List, Generator, Optional
from lxml.html import HtmlElement, fromstring, tostring
from lxml.html.clean import Cleaner
import traceback
from html2text import HTML2Text
import requests
import lxml
from lxml.html import HtmlElement, tostring, fromstring
from fastcore.parallel import parallel
from urllib.parse import  unquote, urlparse
import re

def html_repr(self):
    return lxml.html.tostring(self, encoding='unicode', pretty_print=True)

HtmlElement.__repr__ = html_repr
HtmlElement.__str__ = html_repr

In [7]:
url = "https://or.wikipedia.org/wiki/%E0%AC%AA%E0%AC%A6%E0%AD%8D%E0%AC%AE_%E0%AC%AC%E0%AC%BF%E0%AC%AD%E0%AD%82%E0%AC%B7%E0%AC%A3"
url = unquote(url)
print(f"{url=}")
urlparse(url)

url='https://or.wikipedia.org/wiki/ପଦ୍ମ_ବିଭୂଷଣ'


ParseResult(scheme='https', netloc='or.wikipedia.org', path='/wiki/ପଦ୍ମ_ବିଭୂଷଣ', params='', query='', fragment='')

In [8]:
def remove_newlines(text):
    # Replace newline characters with an empty string
    return text.replace('\n', '')

### WebScraper

In [9]:


# Configure logging
"""
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)
"""
class WebScraper:
    def __init__(self, url: str, base_xpath:str, title_xpath:str):
        """Initialize scraper with URL and extract body."""
        self.url = url
        self.body = self._get_body(base_xpath, title_xpath)

    def _get_body(self, base_xpath:str, title_xpath:str) -> Optional[HtmlElement]:
        """Fetch and clean HTML body from URL."""
        try:
            logger.debug(f"Fetching URL: {self.url}")
            #response = httpx.get(self.url)
            response = requests.get(self.url, timeout=50)
            html_tree = fromstring(response.text)

            self.title = html_tree.xpath(title_xpath)[0].text_content()

            body = html_tree.xpath('//body')[0]
            assert isinstance(body, HtmlElement)
            
            logger.debug("Cleaning HTML body")
            cleaner = Cleaner(javascript=True, style=True)
            cleaned_body = cleaner.clean_html(body)
            
            reconstructed_body = fromstring(''.join(
                tostring(c, encoding='unicode') for c in cleaned_body
            )).xpath(base_xpath)[0]

            logger.info(f"Successfully processed URL: {self.url}")
            return reconstructed_body
        
        except Exception as e:
            logger.error(f"Error processing URL {self.url}: {str(e)}")
            logger.error(traceback.format_exc())
            return None

    def find_elements_by_xpaths(self, xpaths: List[str]) -> Generator[List[HtmlElement], None, None]:
        """Find elements in body using multiple XPaths."""
        if self.body is None:
            logger.error("Body is None, cannot find elements")
            return
        
        for xpath in xpaths:
            try:
                logger.debug(f"Searching for xpath: {xpath}")
                elements = self.body.xpath(xpath)
                logger.info(f"Found {len(elements)} elements for xpath: {xpath}")
                yield elements
            except Exception as e:
                logger.error(f"Error finding elements with xpath {xpath}: {str(e)}")
                logger.error(traceback.format_exc())
                yield []

    def delete_elements_by_xpath(self, xpaths: List[str]) -> None:
        """Delete elements from body using multiple XPaths."""
        if self.body is None:
            logger.error("Body is None, cannot delete elements")
            return
        
        for i, elements in enumerate(self.find_elements_by_xpaths(xpaths)):
            logger.info(f"{xpaths[i]} --> {len(elements)} elements found")
            for element in elements:
                try:
                    parent = element.getparent()
                    if parent is not None:
                        logger.debug(f"Removing element: {remove_newlines(element.text_content())[:50]}...")
                        parent.remove(element)
                except Exception as e:
                    logger.error(f"Error removing element: {str(e)}")
                    logger.error(traceback.format_exc())

    def get_following_siblings(self, element: HtmlElement) -> Generator[HtmlElement, None, None]:
        """Get all following siblings of an element."""
        try:
            next_sibling = element.getnext()
            while next_sibling is not None:
                logger.debug(f"Found sibling: {next_sibling.tag}")
                yield next_sibling
                next_sibling = next_sibling.getnext()
        except Exception as e:
            logger.error(f"Error getting following siblings: {str(e)}")
            logger.error(traceback.format_exc())

    def delete_all_following_elements_by_xpath(self, xpath: str) -> int:
        """Delete all following sibling elements of the element found by xpath."""
        if self.body is None:
            logger.error("Body is None, cannot delete following elements")
            return 0
        
        try:
            element = self.body.xpath(xpath)[0]
            logger.info(f"Found target element: {element.text_content()[:50]}... following count: {len(list(self.get_following_siblings(element)))}")
            
            sibling_following = list(self.get_following_siblings(element))
            removed_count = 0

            for sibling in sibling_following:
                try:
                    sibling.getparent().remove(sibling)
                    removed_count += 1
                    logger.debug(f"Removed sibling: {sibling.text_content()[:20]}...")
                except Exception as e:
                    logger.error(f"Failed to remove sibling: {str(e)}")
                    logger.error(traceback.format_exc())
            
            logger.info(f"Removed {removed_count} following elements")
            return removed_count
        except Exception as e:
            logger.error(f"Error in delete_all_following_elements_by_xpath: {str(e)}")
            logger.error(traceback.format_exc())
            return 0
      
    def to_markdown(self, body_width: int = 500000) -> Optional[str]:
        """Convert the processed HTML body to markdown."""
        if self.body is None:
            logger.error(f"Body is None, cannot convert to markdown for : {self.url}")
            return ''

        try:
            logger.debug("Converting HTML to markdown")
            h2t = HTML2Text(bodywidth=body_width)
            h2t.ignore_links = True
            h2t.mark_code = True
            h2t.ignore_images = True
            
            html_string = tostring(self.body, encoding='unicode')
            markdown_text = h2t.handle(html_string)
            
            logger.info("Successfully converted HTML to markdown")
            return f"# {self.title} \n{markdown_text}"
        except Exception as e:
            logger.error(f"Error converting to markdown: {str(e)} : {self.url}")
            logger.error(traceback.format_exc())
            return None
    def get_processed_body(self) -> Optional[HtmlElement]:
        """Return the processed body."""
        return self.body


In [10]:
xpaths_to_remove = [
    '//div[@class="navbox"]',
    '//div'
    '//a[@class="mw-jump-link"]',
    '//div[@class="vector-header-container"]',
    '//span[@class="plainlinks"]',
    '//nav',
    '//div[@class="vector-page-toolbar"]',
    '//div[@class="title-shortlink-container"]',
    '//div[@id="p-lang-btn"]',
    '//div[@id="siteSub"]',
    '//div[@class="vector-page-toolbar"]',
    '//table[contains(@class, "infobox")]',
    '//span[@class="mw-editsection"]',
    '//div[h2[@id="ଆଧାର"]]',
    '//div[@class="reflist"]',
    '//div[@class="printfooter"]',
    '//div[h2[@id="ବାହାର_ଆଧାର"]]//following::ul',
    '//div[h2[@id="ବାହାର_ଆଧାର"]]',
    '//div[@id="catlinks"]',
    '//div[@class="mw-footer-container"]',
    '//sup',
]
following_xpath = '//div[h2[@id="ଆଧାର"]]'

#scraper.delete_all_following_elements_by_xpath(following_xpath)
#scraper.delete_elements_by_xpath(xpaths_to_remove)

In [11]:
scraper = WebScraper(url, base_xpath='//div[@id="mw-content-text"]', title_xpath='//h1[@id="firstHeading"]')
scraper.delete_all_following_elements_by_xpath(following_xpath)
scraper.delete_elements_by_xpath(xpaths_to_remove)

with open("temp.md", "w") as file:
    file.write(scraper.to_markdown())

In [13]:
def get_md(url):
    scraper = WebScraper(url, base_xpath='//div[@id="mw-content-text"]', title_xpath='//h1[@id="firstHeading"]')
    scraper.delete_all_following_elements_by_xpath(following_xpath)
    scraper.delete_elements_by_xpath(xpaths_to_remove)
    return scraper.to_markdown()
get_md(url),url

('# ପଦ୍ମ ବିଭୂଷଣ \n****\n\n**ପଦ୍ମବିଭୂଷଣ** , ଭାରତ ରତ୍ନ ପରେ ଭାରତୀୟ ଗଣତନ୍ତ୍ରର ଦ୍ୱିତୀୟ ସର୍ବୋଚ୍ଚ ବେସାମରିକ ସମ୍ମାନ ଅଟେ । ୨ ଜାନୁଆରୀ ୧୯୫୪ରେ ସ୍ଥାପିତ ଏହି ପୁରସ୍କାର "ଅସାଧାରଣ ଏବଂ ଉଲ୍ଲେଖନୀୟ ସେବା" ପାଇଁ ପ୍ରଦାନ କରାଯାଏ । ଜାତି, ବୃତ୍ତି, ପଦବୀ କିମ୍ବା ଲିଙ୍ଗ ନିର୍ବିଶେଷରେ ସମସ୍ତ ବ୍ୟକ୍ତି ଏହି ପୁରସ୍କାର ପାଇବାକୁ ଯୋଗ୍ୟ ଅଟନ୍ତି । ତେବେ ଡାକ୍ତର ଓ ବୈଜ୍ଞାନିକଙ୍କ ବ୍ୟତୀତ ରାଷ୍ଟ୍ରାୟତ୍ତ ଉଦ୍ୟୋଗଗୁଡ଼ିକରେ କାର୍ଯ୍ୟ କରୁଥିବା ସରକାରୀ କର୍ମଚାରୀମାନେ ଏହି ପୁରସ୍କାର ପାଇବାକୁ ଯୋଗ୍ୟ ନୁହଁନ୍ତି। ୨୦୨୨ ସୁଦ୍ଧା ୧୯ ଜଣ ମରଣୋତ୍ତର ଓ ୨୧ ଜଣ ଅଣନାଗରିକଙ୍କୁ ମିଶାଇ ୩୨୫ ଜଣଙ୍କୁ ଏହି ପୁରସ୍କାର ପ୍ରଦାନ କରାଯାଇଛି।\n\nପ୍ରତିବର୍ଷ ମେ ୧ରୁ ସେପ୍ଟେମ୍ବର ୧୫ ମଧ୍ୟରେ ଏହି ପୁରସ୍କାର ପାଇଁ ସୁପାରିସ ପଦ୍ମ ପୁରସ୍କାର କମିଟି ନିକଟରେ ଦାଖଲ କରାଯାଏ । ଏହି କମିଟି ଭାରତର ପ୍ରଧାନମନ୍ତ୍ରୀଙ୍କଦ୍ୱାରା ଗଠିତ । ସମସ୍ତ ରାଜ୍ୟ ଓ କେନ୍ଦ୍ରଶାସିତ ଅଞ୍ଚଳ ସରକାର, ଭାରତ ସରକାରଙ୍କ ମନ୍ତ୍ରଣାଳୟ, ଭାରତ ରତ୍ନ ଏବଂ ପୂର୍ବ ପଦ୍ମବିଭୂଷଣ ପୁରସ୍କାର ପ୍ରାପ୍ତ ବ୍ୟକ୍ତି, ଉତ୍କର୍ଷ ପ୍ରତିଷ୍ଠାନ, ମନ୍ତ୍ରୀ, ମୁଖ୍ୟମନ୍ତ୍ରୀ ଓ ରାଜ୍ୟପାଳ ଏବଂ ବ୍ୟକ୍ତିଗତ ବ୍ୟକ୍ତିଙ୍କ ସମେତ ସଂସଦର ସଦସ୍ୟମାନଙ୍କଠାରୁ ଏହି ସୁପାରିସ ଗ୍ରହଣ କରାଯାଇଥାଏ। ପରେ କମିଟି ପରବର୍ତ୍ତୀ ଅନୁମୋଦନ ପାଇଁ ପ୍ରଧାନମନ୍ତ୍ରୀ ଓ ରାଷ୍ଟ୍ରପତିଙ୍କ ନ

### use csv

In [14]:
!ls

content_links.csv	      process_wikipedia_web2md.ipynb
get_all_odia_web_pages.ipynb  temp.md
notebook_logs.log	      wikipedia_data_with_md.csv


In [3]:
import pandas as pd
df = pd.read_csv('content_links.csv')
df.head()

,links
0,https://or.wikipedia.org/wiki/0_(number)
1,https://or.wikipedia.org/wiki/100
2,https://or.wikipedia.org/wiki/1000
3,https://or.wikipedia.org/wiki/1001
4,https://or.wikipedia.org/wiki/1002


In [16]:
links = list(df['links'])
len(links)

38111

In [17]:
import os
os.cpu_count()

4

In [28]:
mds = parallel(get_md, links, progress=True, n_workers=16)

In [32]:
empty, none = [], []

for i, j in zip(links, mds):
    if j == '':
        empty.append(i)
    if j == None:
        none.append(i)
len(empty), len(none)

(0, 0)

In [33]:
data = {
    'links': links,
    'mds': mds
}
data_tf = pd.DataFrame.from_dict(data)
data_tf.head()

,links,mds
0,https://or.wikipedia.org/wiki/0_(number),# ୦ (ସଂଖ୍ୟା) \n****\n\nବାସ୍ତବ ସଂଖ୍ୟା ସମୂହରେ ଶୁ...
1,https://or.wikipedia.org/wiki/100,# ୧୦୦ \n _\n\nଏହି ଲେଖାଟି ୧୦୦ ମସିହା ବାବଦରେ ଅ...
2,https://or.wikipedia.org/wiki/1000,# ୧୦୦୦ \n _\n\nଏହି ଲେଖାଟି ୧୦୦୦ ମସିହା ବାବଦରେ...
3,https://or.wikipedia.org/wiki/1001,# ୧୦୦୧ \n _\n\nଏହି ଲେଖାଟି ୧୦୦୧ ମସିହା ବାବଦରେ...
4,https://or.wikipedia.org/wiki/1002,# ୧୦୦୨ \n _\n\nଏହି ଲେଖାଟି ୧୦୦୨ ମସିହା ବାବଦରେ...


In [ ]:
data_tf.to_csv("wiki_extracted.csv", index=False )

In [ ]:
data_tf.to_csv??

### Extract text

In [4]:
!ls

Untitled.ipynb		      process_wikipedia_web2md.ipynb
content_links.csv	      temp.md
get_all_odia_web_pages.ipynb  wiki_extracted.csv
notebook_logs.log


In [64]:
import pandas as pd
df = pd.read_csv('wiki_extracted.csv')
df.head()

,links,mds
0,https://or.wikipedia.org/wiki/0_(number),# ୦ (ସଂଖ୍ୟା) \n****\n\nବାସ୍ତବ ସଂଖ୍ୟା ସମୂହରେ ଶୁ...
1,https://or.wikipedia.org/wiki/100,# ୧୦୦ \n _\n\nଏହି ଲେଖାଟି ୧୦୦ ମସିହା ବାବଦରେ ଅ...
2,https://or.wikipedia.org/wiki/1000,# ୧୦୦୦ \n _\n\nଏହି ଲେଖାଟି ୧୦୦୦ ମସିହା ବାବଦରେ...
3,https://or.wikipedia.org/wiki/1001,# ୧୦୦୧ \n _\n\nଏହି ଲେଖାଟି ୧୦୦୧ ମସିହା ବାବଦରେ...
4,https://or.wikipedia.org/wiki/1002,# ୧୦୦୨ \n _\n\nଏହି ଲେଖାଟି ୧୦୦୨ ମସିହା ବାବଦରେ...


In [65]:
mds = df['mds']
len(mds) == len(df)

True

We have the markdown string. Extract char only as I am GPU poor.<br>
- A new remove_multiple_chars function that can remove repetitions of any specified characters beyond a certain threshold.
- A clean_text function that combines markdown removal and multiple character removal.
- Customizable set of characters to remove multiple occurrences of.
- Adjustable maximum number of allowed repetitions for characters.

And there are multiple file like one extracted from [here](https://or.wikipedia.org/wiki/୧୦୦). Which is generated via bot it contains string like 
```
ଏହି ଲେଖାଟି ବଟ୍ ବ୍ୟବହାର କରି ତିଆରି କରାଯାଇଛି ଏବଂ ଏହା ପର୍ଯ୍ୟାୟ କ୍ରମେ ଉନ୍ନତି ହେଉଅଛି । ଏଣୁ ଏହି ଛାଞ୍ଚ ନଉଠିବା ପର୍ଯ୍ୟନ୍ତ କୌଣସି ବଦଳ କରନ୍ତୁ ନାହିଁ । ଯଦି ଆପଣ କୌଣସି ଗୁରୁତ୍ତ୍ୱପୂର୍ଣ୍ଣ ତ୍ରୁଟି ଆପଣଙ୍କ ଦୃଷ୍ଟିଗୋଚର ହେଉଅଛି ତେବେ ପୃଷ୍ଠା ଆଲୋଚନା‌ରେ ସୂଚିତ କରନ୍ତୁ ।
```
Removing tables as most of the times it might contain `name` or `place` or some words. So removing the tables completely.

In [123]:
import re
from functools import reduce

bot_msg = 'ଏହି ଲେଖାଟି ବଟ୍ ବ୍ୟବହାର କରି ତିଆରି କରାଯାଇଛି ଏବଂ ଏହା ପର୍ଯ୍ୟାୟ କ୍ରମେ ଉନ୍ନତି ହେଉଅଛି । ଏଣୁ ଏହି ଛାଞ୍ଚ ନଉଠିବା ପର୍ଯ୍ୟନ୍ତ କୌଣସି ବଦଳ କରନ୍ତୁ ନାହିଁ । ଯଦି ଆପଣ କୌଣସି ଗୁରୁତ୍ତ୍ୱପୂର୍ଣ୍ଣ ତ୍ରୁଟି ଆପଣଙ୍କ ଦୃଷ୍ଟିଗୋଚର ହେଉଅଛି ତେବେ ପୃଷ୍ଠା ଆଲୋଚନା‌ରେ ସୂଚିତ କରନ୍ତୁ ।'

MARKDOWN_PATTERNS = [
    (re.compile(r'^#+\s*', re.MULTILINE), ''),  # Headers
    (re.compile(r'\*\*(.*?)\*\*'), r'\1'),  # Bold text
    (re.compile(r'`(.*?)`'), r'\1'),  # Inline code
    (re.compile(r'```[\s\S]*?```'), ''),  # Code blocks
    (re.compile(r'^\s*\*\s+', re.MULTILINE), ''),  # List items
    (re.compile(r'^_+$', re.MULTILINE), ''),  # Horizontal rules with underscores
    (re.compile(r'^[^\n]*\|[^\n]*\n', re.MULTILINE), ''),  # Remove table rows (lines with pipes inside)
    (re.compile(r'^\s*[-:| ]+\s*\n', re.MULTILINE), ''),   # Remove table headers and separators
    (re.compile(r'\n+'), r'\n'),  # Remove multiple newlines
    
    
]

def remove_markdown(text):
    return reduce(lambda t, pattern: pattern[0].sub(pattern[1], t), MARKDOWN_PATTERNS, text)

def remove_multiple_chars(text, chars_to_remove=None, max_repetitions=1):
    if chars_to_remove is None:
        # Default set of characters to remove multiple occurrences of
        chars_to_remove = r'\s!@#$%^&*()_+-=[]{}|;:,.<>?/~`'
    
    # Escape special regex characters
    escaped_chars = re.escape(chars_to_remove)
    
    # Create a pattern to match multiple occurrences of any character in the set
    pattern = f'([{escaped_chars}])\\1{{{max_repetitions},}}'
    
    # Replace multiple occurrences with the specified number of repetitions
    return re.sub(pattern, lambda m: m.group(1) * max_repetitions, text)

def clean_text(text, chars_to_remove=None, max_repetitions=1):
    if bot_msg in text:
        return ''
    # Remove markdown
    cleaned = remove_markdown(text)
    
    # Remove multiple unwanted characters
    cleaned = remove_multiple_chars(cleaned, chars_to_remove, max_repetitions)
    
    # Strip leading/trailing whitespace from each line and the entire text
    cleaned = '\n'.join(line.strip() for line in cleaned.split('\n')).strip()
    
    return cleaned 


In [125]:
ext_txt =  parallel(clean_text, mds, progress=True, n_workers=16)

In [130]:
data = {
    'link': (df['links']),
    'text': ext_txt
}
data_df = pd.DataFrame.from_dict(data)
data_df.head()
data_df.to_csv("odia_wiki.csv", index=False )

In [131]:
data_df.head()

,link,text
0,https://or.wikipedia.org/wiki/0_(number),୦ (ସଂଖ୍ୟା)\nବାସ୍ତବ ସଂଖ୍ୟା ସମୂହରେ ଶୁନ୍ୟ (୦) ହେଉ...
1,https://or.wikipedia.org/wiki/100,୧୦୦\n_\nଏହି ଲେଖାଟି ୧୦୦ ମସିହା ବାବଦରେ ଅଟେ । ସଂଖ୍...
2,https://or.wikipedia.org/wiki/1000,
3,https://or.wikipedia.org/wiki/1001,
4,https://or.wikipedia.org/wiki/1002,


In [57]:
data_tf.iloc[2]['text']

''

In [132]:
 (data_tf['text'] == '').sum()

2848

In [133]:
data_tf.iloc[336]['text']

'୧୨ଶ ଲୋକ ସଭା\nଭାରତୀୟ ସଂସଦର ତଳ ସଦନ ଲୋକ ସଭାର ୧୨ଶ ଲୋକ ସଭା କୁ ୧୯୯୮ ମସିହା ସାଧାରଣ ନିର୍ବାଚନରେ ନିର୍ବାଚିତ ସଭ୍ୟମାନେ ଯାଇଥିଲେ । ଏହି ଲୋକ ସଭାର ସମୟସୀମା ଥିଲା ୧୦ ମାର୍ଚ୍ଚ ୧୯୯୮ରୁ ୨୬ ଅପ୍ରେଲ ୧୯୯୯ । ୧୯୯୮ ସାଧାରଣ ନିର୍ବାଚନ ଫେବୃଆରୀ-ମାର୍ଚ୍ଚ ୧୯୯୮ରେ ଅନୁଷ୍ଠିତ ହୋଇଥିଲା । ଏହି ଲୋକ ସଭା ସାଧାରଣ ନିର୍ବାଚନ ପରେ ସଂସଦର ରାଜ୍ୟ ସଭାରେ ଥିବା ୯ ଜଣ ସାଂସଦ ଲୋସଭାକୁ ନିର୍ବାଚିତ ହୋଇଥିଲେ ।\nଏହି ଲୋକ ସଭା ନିର୍ବାଚନରେ ପୂର୍ବ ଦୁଇ ଲୋକ ସଭା ୧୦ମ ଲୋକ ସଭା ଓ ୧୧ଶ ଲୋକ ସଭା ଭଳି ଝୁଲା ସରକାର ହୋଇଥିଲା, କାରଣ କୌଣସି ନିର୍ଦ୍ଦିଷ୍ଟ ଦଳ ସଂଖ୍ୟା ଗରିଷ୍ଠତା ହାସଲ କରିନଥିଲେ । ଅଟଳ ବିହାରୀ ବାଜପେୟୀ ଏହି ଲୋକ ସଭାର ପ୍ରଧାନମନ୍ତ୍ରୀ ହୋଇଥିଲେ, ମାତ୍ର ସ୍ପଷ୍ଟ ଜନାଦେଶ ନ ହେବାରୁ ତାଙ୍କର ସରକାର ମାତ୍ର ୧୩ ମାସ ଚାଲିଥିଲା । ଏ.ଆଇ.ଏ ଡି.ଏମ୍.କେର ସମର୍ଥନ ପ୍ରତ୍ୟାହାର ପରେ ଅନ୍ୟାନ୍ୟ ରାଜନୈତିକ ଦଳମାନେ ସଂଖ୍ୟାଗରିଷ୍ଠତା ପାଇଁ ସମର୍ଥନ ଦେଇ ନଥିଲେ ।କେ. ଆର. ନାରାୟଣନ ଲୋକ ସଭାର ବିରୋଧୀ ଦଳର ନେତା ସୋନିଆ ଗାନ୍ଧୀଙ୍କୁ ସରକାର ଗଠନ ପାଇଁ ଆମନ୍ତ୍ରଣ ଦେଇଥିଲେ, ମାତ୍ର ସେମାନେ ମଧ୍ୟ ସଂଖ୍ୟା ଗରିଷ୍ଠତା ପାଇଁ ସମର୍ଥନ ପାଇନଥିଲେ । ଫଳରେ ରାଷ୍ଟ୍ରପତି ଏହି ଲୋକ ସଭାକୁ ଭାଙ୍ଗିଦେଇଥିଲେ । ଏହାପରେ ୧୯୯୯ ମସିହାରେ ଦେଶରେ ୧୩ଶ ଲୋକ ସଭା ପାଇଁ ସାଧାରଣ ନିର୍ବାଚନ ଅନୁଷ୍ଠିତ ହୋଇଥିଲା, ଯେତେବେଳେ ଏକ ସ୍ଥାୟୀ ସର

In [134]:
data_tf.iloc[336]['links']

'https://or.wikipedia.org/wiki/12th_Lok_Sabha'

In [135]:
df.iloc[336]['mds']

'# ୧୨ଶ ଲୋକ ସଭା \n****\n\n  \nଭାରତୀୟ ସଂସଦର ତଳ ସଦନ ଲୋକ ସଭାର **୧୨ଶ ଲୋକ ସଭା** କୁ ୧୯୯୮ ମସିହା ସାଧାରଣ ନିର୍ବାଚନରେ ନିର୍ବାଚିତ ସଭ୍ୟମାନେ ଯାଇଥିଲେ । ଏହି ଲୋକ ସଭାର ସମୟସୀମା ଥିଲା ୧୦ ମାର୍ଚ୍ଚ ୧୯୯୮ରୁ ୨୬ ଅପ୍ରେଲ ୧୯୯୯ । ୧୯୯୮ ସାଧାରଣ ନିର୍ବାଚନ ଫେବୃଆରୀ-ମାର୍ଚ୍ଚ ୧୯୯୮ରେ ଅନୁଷ୍ଠିତ ହୋଇଥିଲା । ଏହି ଲୋକ ସଭା ସାଧାରଣ ନିର୍ବାଚନ ପରେ ସଂସଦର ରାଜ୍ୟ ସଭାରେ ଥିବା ୯ ଜଣ ସାଂସଦ ଲୋସଭାକୁ ନିର୍ବାଚିତ ହୋଇଥିଲେ ।\n\nଏହି ଲୋକ ସଭା ନିର୍ବାଚନରେ ପୂର୍ବ ଦୁଇ ଲୋକ ସଭା ୧୦ମ ଲୋକ ସଭା ଓ ୧୧ଶ ଲୋକ ସଭା ଭଳି ଝୁଲା ସରକାର ହୋଇଥିଲା, କାରଣ କୌଣସି ନିର୍ଦ୍ଦିଷ୍ଟ ଦଳ ସଂଖ୍ୟା ଗରିଷ୍ଠତା ହାସଲ କରିନଥିଲେ । ଅଟଳ ବିହାରୀ ବାଜପେୟୀ ଏହି ଲୋକ ସଭାର ପ୍ରଧାନମନ୍ତ୍ରୀ ହୋଇଥିଲେ, ମାତ୍ର ସ୍ପଷ୍ଟ ଜନାଦେଶ ନ ହେବାରୁ ତାଙ୍କର ସରକାର ମାତ୍ର ୧୩ ମାସ ଚାଲିଥିଲା । ଏ.ଆଇ.ଏ ଡି.ଏମ୍.କେର ସମର୍ଥନ ପ୍ରତ୍ୟାହାର ପରେ ଅନ୍ୟାନ୍ୟ ରାଜନୈତିକ ଦଳମାନେ ସଂଖ୍ୟାଗରିଷ୍ଠତା ପାଇଁ ସମର୍ଥନ ଦେଇ ନଥିଲେ ।କେ. ଆର. ନାରାୟଣନ ଲୋକ ସଭାର ବିରୋଧୀ ଦଳର ନେତା ସୋନିଆ ଗାନ୍ଧୀଙ୍କୁ ସରକାର ଗଠନ ପାଇଁ ଆମନ୍ତ୍ରଣ ଦେଇଥିଲେ, ମାତ୍ର ସେମାନେ ମଧ୍ୟ ସଂଖ୍ୟା ଗରିଷ୍ଠତା ପାଇଁ ସମର୍ଥନ ପାଇନଥିଲେ । ଫଳରେ ରାଷ୍ଟ୍ରପତି ଏହି ଲୋକ ସଭାକୁ ଭାଙ୍ଗିଦେଇଥିଲେ । ଏହାପରେ ୧୯୯୯ ମସିହାରେ ଦେଶରେ ୧୩ଶ ଲୋକ ସଭା ପାଇଁ ସାଧାରଣ ନିର୍ବାଚନ ଅନୁଷ୍ଠିତ ହୋଇଥିଲା, 

In [136]:
data_tf.index[data_tf['links'] == 'https://or.wikipedia.org/wiki/୧୦୦୦']

Index([35642], dtype='int64')

In [137]:
data_tf.iloc[35642]['text']

''